In [29]:
import pandas as pd
import numpy as np
import configparser
from ast import literal_eval


config = configparser.ConfigParser()
config.read('targets_of_interest.ini')
targets = literal_eval(config['section1']['targets_of_interest'])


In [30]:
behaviors = pd.read_csv('behaviors_wide_deID.csv',index_col=0)
behaviors_orig = behaviors.copy()

In [31]:
cols = behaviors.columns.tolist()
target_ndr_cols = [col for col in cols if 'Behavior_No_Data_Recorded_' in col]

In [32]:
behaviors.info()

<class 'pandas.core.frame.DataFrame'>
Index: 77793 entries, 0 to 77792
Data columns (total 21 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Concat                                77793 non-null  object 
 1   n_observations                        77793 non-null  int64  
 2   Missing_Shift                         77793 non-null  int64  
 3   Behavior_aggression                   77793 non-null  int64  
 4   Behavior_sib                          77793 non-null  int64  
 5   Behavior_elopement                    77793 non-null  int64  
 6   Behavior_disruptive                   77793 non-null  int64  
 7   Behavior_other                        77793 non-null  int64  
 8   Behavior_Episodes_aggression          77793 non-null  float64
 9   Behavior_Episodes_sib                 77793 non-null  float64
 10  Behavior_Episodes_elopement           77793 non-null  float64
 11  Behavior_Episodes_di

In [33]:
# This means nothing was recorded at all for the shift
behaviors[(behaviors[target_ndr_cols].sum(axis=1) == 0) & (behaviors['Behavior_No_Data_Recorded'] > 0)]


,Concat,n_observations,Missing_Shift,Behavior_aggression,Behavior_sib,Behavior_elopement,Behavior_disruptive,Behavior_other,Behavior_Episodes_aggression,Behavior_Episodes_sib,...,Behavior_Episodes_disruptive,Behavior_Episodes_other,Behavior_LOA,Behavior_None,Behavior_No_Data_Recorded,Behavior_No_Data_Recorded_aggression,Behavior_No_Data_Recorded_sib,Behavior_No_Data_Recorded_elopement,Behavior_No_Data_Recorded_disruptive,Behavior_No_Data_Recorded_other
21,1 2019-09-22 07:00:00,1,0,0,0,0,0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
22,1 2019-09-22 15:00:00,1,0,0,0,0,0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
37,1 2019-09-27 15:00:00,1,0,0,0,0,0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
58,1 2019-10-04 15:00:00,1,0,0,0,0,0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
81,1 2019-10-12 07:00:00,1,0,0,0,0,0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77695,9 2020-07-30 15:00:00,1,1,0,0,0,0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
77696,9 2020-07-30 23:00:00,1,1,0,0,0,0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
77697,9 2020-07-31 07:00:00,1,1,0,0,0,0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
77698,9 2020-07-31 15:00:00,1,1,0,0,0,0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [34]:
# If there's nothing recorded at all, set no data recorded
behaviors = behaviors.set_index('Concat')
behaviors.loc[behaviors.sum(axis=1) == 0,'Behavior_No_Data_Recorded'] = 1.0

In [35]:
no_data_cols = ['Behavior_No_Data_Recorded_' + targ for targ in targets]

In [36]:
# Basically if it's an "other" no data recorded *only*, then it's not an issue
# Unless it's the only observation!
behaviors.loc[(behaviors['n_observations'] > 1) & (behaviors['Behavior_No_Data_Recorded_other']>0) & (behaviors[no_data_cols].sum(axis=1)==0),'Behavior_No_Data_Recorded'] = 0

In [37]:
behaviors = behaviors.drop('n_observations',axis=1)

In [38]:
behaviors = behaviors.reset_index()
ids = behaviors['Concat'].str.split(n=2,expand=True)
ids = ids[0] + " " + ids[1]

In [39]:
behaviors[behaviors['Behavior_No_Data_Recorded_aggression'] > 0]

,Concat,Missing_Shift,Behavior_aggression,Behavior_sib,Behavior_elopement,Behavior_disruptive,Behavior_other,Behavior_Episodes_aggression,Behavior_Episodes_sib,Behavior_Episodes_elopement,Behavior_Episodes_disruptive,Behavior_Episodes_other,Behavior_LOA,Behavior_None,Behavior_No_Data_Recorded,Behavior_No_Data_Recorded_aggression,Behavior_No_Data_Recorded_sib,Behavior_No_Data_Recorded_elopement,Behavior_No_Data_Recorded_disruptive,Behavior_No_Data_Recorded_other
2448,11 2019-12-20 07:00:00,0,0,0,0,1,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,1.0,0.0,1.0,1.0,0.0
4543,13 2020-02-17 15:00:00,0,1,1,0,0,0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,1.0,0.0,1.0,0.0
4974,13 2020-07-10 07:00:00,0,0,1,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,1.0,0.0,1.0,0.0
5994,14 2020-07-18 07:00:00,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
6017,14 2020-07-25 23:00:00,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60330,61 2019-10-14 07:00:00,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
60687,61 2020-02-10 07:00:00,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
60783,61 2020-03-13 07:00:00,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
69405,7 2020-03-13 07:00:00,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0


In [40]:
behaviors['Concat_Date'] = ids

In [41]:
behaviors[['Person_ID', 'Date','Shift']] = behaviors['Concat'].str.split(' ',expand =True)
by_shift_pivoted = behaviors.pivot(index=['Person_ID','Date'], columns='Shift').reset_index(level=0)
by_shift_pivoted = by_shift_pivoted.reset_index().drop('Concat',axis=1)
by_shift_pivoted.columns = ["".join(pair) for pair in by_shift_pivoted.columns]
by_shift_pivoted

C:\Users\jen-f\AppData\Local\Temp\ipykernel_65216\2424302537.py:3: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  by_shift_pivoted = by_shift_pivoted.reset_index().drop('Concat',axis=1)


,Date,Person_ID,Missing_Shift07:00:00,Missing_Shift15:00:00,Missing_Shift23:00:00,Behavior_aggression07:00:00,Behavior_aggression15:00:00,Behavior_aggression23:00:00,Behavior_sib07:00:00,Behavior_sib15:00:00,...,Behavior_No_Data_Recorded_elopement23:00:00,Behavior_No_Data_Recorded_disruptive07:00:00,Behavior_No_Data_Recorded_disruptive15:00:00,Behavior_No_Data_Recorded_disruptive23:00:00,Behavior_No_Data_Recorded_other07:00:00,Behavior_No_Data_Recorded_other15:00:00,Behavior_No_Data_Recorded_other23:00:00,Concat_Date07:00:00,Concat_Date15:00:00,Concat_Date23:00:00
0,2019-09-15,1,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1 2019-09-15,1 2019-09-15,1 2019-09-15
1,2019-09-16,1,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1 2019-09-16,1 2019-09-16,1 2019-09-16
2,2019-09-17,1,0,0,0,0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1 2019-09-17,1 2019-09-17,1 2019-09-17
3,2019-09-18,1,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1 2019-09-18,1 2019-09-18,1 2019-09-18
4,2019-09-19,1,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1 2019-09-19,1 2019-09-19,1 2019-09-19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25926,2020-08-27,9,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9 2020-08-27,9 2020-08-27,9 2020-08-27
25927,2020-08-28,9,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9 2020-08-28,9 2020-08-28,9 2020-08-28
25928,2020-08-29,9,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9 2020-08-29,9 2020-08-29,9 2020-08-29
25929,2020-08-30,9,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9 2020-08-30,9 2020-08-30,9 2020-08-30


In [42]:
times = ['07:00:00','15:00:00','23:00:00']
for target in targets:
    for time in times:
        by_shift_pivoted['Behavior_'+target+time] = np.where(by_shift_pivoted['Behavior_No_Data_Recorded_'+target+time] > 0,
                                                        float("NaN"),
                                                        by_shift_pivoted['Behavior_'+target+time])
        by_shift_pivoted['Behavior_Episodes_'+target+time] = np.where(by_shift_pivoted['Behavior_No_Data_Recorded_'+target+time] > 0,
                                                        float("NaN"),
                                                        by_shift_pivoted['Behavior_Episodes_'+target+time])

In [43]:
high_episode_individuals = pd.read_csv('high_episode_flag_ids.csv')['Person_ID'].tolist()
high_episode_individuals

[nan, 21.0, 22.0, 62.0]

In [44]:
episode_cols = [col for col in by_shift_pivoted.columns if 'Episodes' in col]
episode_cols

by_shift_pivoted

for target in targets:
    for time in times:
        for ind in high_episode_individuals:
            print(by_shift_pivoted.loc[(by_shift_pivoted['Behavior_Episodes_' + target +time] > 15) & (by_shift_pivoted['Person_ID']==ind),
                                       ['Person_ID','Behavior_Episodes_' + target +time]])
            by_shift_pivoted.loc[(by_shift_pivoted['Behavior_Episodes_' + target +time] > 15) & (by_shift_pivoted['Person_ID']==ind),'Behavior_No_Data_Recorded_'+target+time] = 1.0

Empty DataFrame
Columns: [Person_ID, Behavior_Episodes_aggression07:00:00]
Index: []
Empty DataFrame
Columns: [Person_ID, Behavior_Episodes_aggression07:00:00]
Index: []
Empty DataFrame
Columns: [Person_ID, Behavior_Episodes_aggression07:00:00]
Index: []
Empty DataFrame
Columns: [Person_ID, Behavior_Episodes_aggression07:00:00]
Index: []
Empty DataFrame
Columns: [Person_ID, Behavior_Episodes_aggression15:00:00]
Index: []
Empty DataFrame
Columns: [Person_ID, Behavior_Episodes_aggression15:00:00]
Index: []
Empty DataFrame
Columns: [Person_ID, Behavior_Episodes_aggression15:00:00]
Index: []
Empty DataFrame
Columns: [Person_ID, Behavior_Episodes_aggression15:00:00]
Index: []
Empty DataFrame
Columns: [Person_ID, Behavior_Episodes_aggression23:00:00]
Index: []
Empty DataFrame
Columns: [Person_ID, Behavior_Episodes_aggression23:00:00]
Index: []
Empty DataFrame
Columns: [Person_ID, Behavior_Episodes_aggression23:00:00]
Index: []
Empty DataFrame
Columns: [Person_ID, Behavior_Episodes_aggression

In [45]:
# by_shift_pivoted[by_shift_pivoted['Behavior_E

In [46]:
concats = []
for target in targets:
    for time in times:
        
        concat = by_shift_pivoted.loc[(by_shift_pivoted['Behavior_Episodes_' + target +time] > 15) ,
                                       ['Person_ID','Date']]
        concats.extend(concat.Person_ID + " " + concat.Date)
        # by_shift_pivoted.loc[(by_shift_pivoted['Behavior_Episodes_' + target +time] > 15) ,'Behavior_No_Data_Recorded_'+target+time] = 1.0
pd.DataFrame({'high_eps_final':sorted(concats)}).to_csv('high_eps_finalized.csv')

In [47]:
inds = []
for target in targets:
    for time in times:    
        inds.extend( by_shift_pivoted.loc[(by_shift_pivoted['Behavior_Episodes_' + target +time] > 15),
                                          'Person_ID'].unique().tolist() )
pd.DataFrame({'Person_ID':sorted(list(set(inds)))}).to_csv('high_ep_unfiltered.csv')

In [48]:
by_shift_pivoted[by_shift_pivoted['Behavior_No_Data_Recorded_aggression07:00:00'] > 0]

,Date,Person_ID,Missing_Shift07:00:00,Missing_Shift15:00:00,Missing_Shift23:00:00,Behavior_aggression07:00:00,Behavior_aggression15:00:00,Behavior_aggression23:00:00,Behavior_sib07:00:00,Behavior_sib15:00:00,...,Behavior_No_Data_Recorded_elopement23:00:00,Behavior_No_Data_Recorded_disruptive07:00:00,Behavior_No_Data_Recorded_disruptive15:00:00,Behavior_No_Data_Recorded_disruptive23:00:00,Behavior_No_Data_Recorded_other07:00:00,Behavior_No_Data_Recorded_other15:00:00,Behavior_No_Data_Recorded_other23:00:00,Concat_Date07:00:00,Concat_Date15:00:00,Concat_Date23:00:00
816,2019-12-20,11,0,0,0,NaN,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,11 2019-12-20,11 2019-12-20,11 2019-12-20
1658,2020-07-10,13,0,0,0,NaN,1.0,0.0,NaN,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,13 2020-07-10,13 2020-07-10,13 2020-07-10
1998,2020-07-18,14,0,0,0,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14 2020-07-18,14 2020-07-18,14 2020-07-18
2027,2020-08-16,14,0,0,0,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14 2020-08-16,14 2020-08-16,14 2020-08-16
2082,2019-10-16,15,0,0,0,NaN,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,15 2019-10-16,15 2019-10-16,15 2019-10-16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20110,2019-10-14,61,0,0,0,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,61 2019-10-14,61 2019-10-14,61 2019-10-14
20229,2020-02-10,61,0,0,0,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,61 2020-02-10,61 2020-02-10,61 2020-02-10
20261,2020-03-13,61,0,0,0,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,61 2020-03-13,61 2020-03-13,61 2020-03-13
23135,2020-03-13,7,0,0,0,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7 2020-03-13,7 2020-03-13,7 2020-03-13


In [49]:
by_shift_pivoted[by_shift_pivoted['Behavior_aggression07:00:00'].isna() & ~by_shift_pivoted['Behavior_sib07:00:00'].isna()]

,Date,Person_ID,Missing_Shift07:00:00,Missing_Shift15:00:00,Missing_Shift23:00:00,Behavior_aggression07:00:00,Behavior_aggression15:00:00,Behavior_aggression23:00:00,Behavior_sib07:00:00,Behavior_sib15:00:00,...,Behavior_No_Data_Recorded_elopement23:00:00,Behavior_No_Data_Recorded_disruptive07:00:00,Behavior_No_Data_Recorded_disruptive15:00:00,Behavior_No_Data_Recorded_disruptive23:00:00,Behavior_No_Data_Recorded_other07:00:00,Behavior_No_Data_Recorded_other15:00:00,Behavior_No_Data_Recorded_other23:00:00,Concat_Date07:00:00,Concat_Date15:00:00,Concat_Date23:00:00
816,2019-12-20,11,0,0,0,NaN,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,11 2019-12-20,11 2019-12-20,11 2019-12-20
1998,2020-07-18,14,0,0,0,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14 2020-07-18,14 2020-07-18,14 2020-07-18
2027,2020-08-16,14,0,0,0,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14 2020-08-16,14 2020-08-16,14 2020-08-16
2082,2019-10-16,15,0,0,0,NaN,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,15 2019-10-16,15 2019-10-16,15 2019-10-16
2178,2020-01-20,15,0,0,0,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15 2020-01-20,15 2020-01-20,15 2020-01-20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20110,2019-10-14,61,0,0,0,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,61 2019-10-14,61 2019-10-14,61 2019-10-14
20229,2020-02-10,61,0,0,0,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,61 2020-02-10,61 2020-02-10,61 2020-02-10
20261,2020-03-13,61,0,0,0,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,61 2020-03-13,61 2020-03-13,61 2020-03-13
23135,2020-03-13,7,0,0,0,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7 2020-03-13,7 2020-03-13,7 2020-03-13


In [50]:
behaviors_orig.columns

Index(['Concat', 'n_observations', 'Missing_Shift', 'Behavior_aggression',
       'Behavior_sib', 'Behavior_elopement', 'Behavior_disruptive',
       'Behavior_other', 'Behavior_Episodes_aggression',
       'Behavior_Episodes_sib', 'Behavior_Episodes_elopement',
       'Behavior_Episodes_disruptive', 'Behavior_Episodes_other',
       'Behavior_LOA', 'Behavior_None', 'Behavior_No_Data_Recorded',
       'Behavior_No_Data_Recorded_aggression', 'Behavior_No_Data_Recorded_sib',
       'Behavior_No_Data_Recorded_elopement',
       'Behavior_No_Data_Recorded_disruptive',
       'Behavior_No_Data_Recorded_other'],
      dtype='object')

In [51]:
behaviors_orig.head(60)

,Concat,n_observations,Missing_Shift,Behavior_aggression,Behavior_sib,Behavior_elopement,Behavior_disruptive,Behavior_other,Behavior_Episodes_aggression,Behavior_Episodes_sib,...,Behavior_Episodes_disruptive,Behavior_Episodes_other,Behavior_LOA,Behavior_None,Behavior_No_Data_Recorded,Behavior_No_Data_Recorded_aggression,Behavior_No_Data_Recorded_sib,Behavior_No_Data_Recorded_elopement,Behavior_No_Data_Recorded_disruptive,Behavior_No_Data_Recorded_other
0,1 2019-09-15 07:00:00,1,0,0,0,0,0,0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1 2019-09-15 15:00:00,1,0,0,0,0,0,0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1 2019-09-15 23:00:00,5,0,0,0,0,0,0,0.0,0.0,...,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1 2019-09-16 07:00:00,5,0,0,0,0,0,0,0.0,0.0,...,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1 2019-09-16 15:00:00,5,0,0,0,0,0,0,0.0,0.0,...,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1 2019-09-16 23:00:00,5,0,0,0,0,0,0,0.0,0.0,...,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0
6,1 2019-09-17 07:00:00,5,0,0,0,0,0,0,0.0,0.0,...,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0
7,1 2019-09-17 15:00:00,3,0,0,1,0,1,0,0.0,1.0,...,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
8,1 2019-09-17 23:00:00,5,0,0,0,0,0,0,0.0,0.0,...,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0
9,1 2019-09-18 07:00:00,5,0,0,0,0,0,0,0.0,0.0,...,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0


In [52]:
by_shift_pivoted.loc[by_shift_pivoted['Behavior_No_Data_Recorded_aggression15:00:00'] > 0]

,Date,Person_ID,Missing_Shift07:00:00,Missing_Shift15:00:00,Missing_Shift23:00:00,Behavior_aggression07:00:00,Behavior_aggression15:00:00,Behavior_aggression23:00:00,Behavior_sib07:00:00,Behavior_sib15:00:00,...,Behavior_No_Data_Recorded_elopement23:00:00,Behavior_No_Data_Recorded_disruptive07:00:00,Behavior_No_Data_Recorded_disruptive15:00:00,Behavior_No_Data_Recorded_disruptive23:00:00,Behavior_No_Data_Recorded_other07:00:00,Behavior_No_Data_Recorded_other15:00:00,Behavior_No_Data_Recorded_other23:00:00,Concat_Date07:00:00,Concat_Date15:00:00,Concat_Date23:00:00
1514,2020-02-17,13,0,0,0,1.0,NaN,0.0,1.0,NaN,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,13 2020-02-17,13 2020-02-17,13 2020-02-17
2444,2019-10-18,16,0,0,0,1.0,NaN,0.0,1.0,NaN,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,16 2019-10-18,16 2019-10-18,16 2019-10-18
2491,2019-12-04,16,0,0,0,0.0,NaN,0.0,1.0,NaN,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,16 2019-12-04,16 2019-12-04,16 2019-12-04
2820,2019-11-03,17,0,0,0,0.0,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17 2019-11-03,17 2019-11-03,17 2019-11-03
6396,2019-10-10,26,0,0,0,0.0,NaN,0.0,0.0,NaN,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,26 2019-10-10,26 2019-10-10,26 2019-10-10
6566,2020-03-28,26,0,0,0,1.0,NaN,0.0,0.0,NaN,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,26 2020-03-28,26 2020-03-28,26 2020-03-28
7809,2020-09-07,29,0,0,0,0.0,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,29 2020-09-07,29 2020-09-07,29 2020-09-07
8516,2020-08-25,30,0,0,0,1.0,NaN,0.0,1.0,NaN,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,30 2020-08-25,30 2020-08-25,30 2020-08-25
8955,2019-11-18,32,0,0,0,1.0,NaN,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,32 2019-11-18,32 2019-11-18,32 2019-11-18
9529,2020-06-19,33,0,0,0,0.0,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33 2020-06-19,33 2020-06-19,33 2020-06-19


In [53]:
for target in targets:
    for time in times:
        by_shift_pivoted.loc[by_shift_pivoted['Behavior_LOA' + time] > 0, 'Behavior_' +target + time] = float("NaN")
        by_shift_pivoted.loc[by_shift_pivoted['Behavior_LOA' + time] > 0, 'Behavior_Episodes_' +target+ time] = float("NaN")
#         by_shift_pivoted.loc[by_shift_pivoted['Behavior_No_Data_Recorded' + time] > 0, 'Behavior_' +target + time] = float("NaN")
#         by_shift_pivoted.loc[by_shift_pivoted['Behavior_No_Data_Recorded' + time] > 0, 'Behavior_Episodes_' +target+ time] = float("NaN")

In [54]:
for target in targets:
    cols = ['Behavior_'+target+time for time in times]
    by_shift_pivoted['Behavior_'+target] = by_shift_pivoted[cols].sum(axis=1,min_count=3) 
    by_shift_pivoted['Behavior_'+target] = np.where(by_shift_pivoted['Behavior_'+target].isna(), float("NaN"), (by_shift_pivoted['Behavior_'+target] > 0) * 1.0)
    cols = ['Behavior_Episodes_'+target+time for time in times]
    by_shift_pivoted['Behavior_Episodes_'+target] = by_shift_pivoted[cols].sum(axis=1, min_count=3)
    by_shift_pivoted['Behavior_Episodes_'+target] = np.where(by_shift_pivoted['Behavior_Episodes_'+target].isna(), float("NaN"), by_shift_pivoted['Behavior_Episodes_'+target])

In [55]:
final_cols = ['Person_ID','Date'] + ['Behavior_'+target for target in targets] + ['Behavior_Episodes_'+target for target in targets]
by_shift_pivoted = by_shift_pivoted[final_cols]

In [56]:
by_shift_pivoted.to_csv('behaviors_pivoted_deID.csv')